### Setup Data Lake mount

Provide information for service principal (created in AD via app registration)

Reference: https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/azure-datalake-gen2#--mount-an-azure-data-lake-storage-gen2-account-using-a-service-principal-and-oauth-20

In [3]:
# Change variables below:
client_id = "put your client id here"
client_secret = "put your client secret here"      # or use dbutils.secrets.get(scope="your scope",key="your scope key to the client secret")
directory_id = "put your directory id here"
adls_account = "put your account name here"        # DO NOT include suffixes such as dfs.core.windows.net
adls_filesystem = "put your filesystem name here"  # create your filesystem prior to running this code 

Mount Data Lake Gen2 for Databricks access

In [5]:
# prepare configs
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": client_id,
           "fs.azure.account.oauth2.client.secret": client_secret,
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{directory_id}/oauth2/token"}

# mount
dbutils.fs.mount(
  source = f"abfss://{adls_filesystem}@{adls_account}.dfs.core.windows.net/",
  mount_point = "/mnt/lake/raw",
  extra_configs = configs)

Out[3]: True

Check if mount is accessing properly

In [7]:
dbutils.fs.ls("/mnt/lake/raw")
#dbutils.fs.unmount("/mnt/lake/raw")

Out[4]: [FileInfo(path='dbfs:/mnt/lake/raw/raw.events/', name='raw.events/', size=0)]

Install event hubs connector library in databricks cluster

In [9]:
# install library in databricks cluster
# https://docs.microsoft.com/en-us/azure/databricks/spark/latest/structured-streaming/streaming-event-hubs